In [299]:
from bs4 import BeautifulSoup as bs
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup
from requests import get
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from googlemaps import Client as GoogleMaps
import time
from time import sleep

from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry


print('Libraries imported.')

Libraries imported.


In [103]:
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
sapo = "https://www.apartments.com/off-campus-housing/il/chicago/university-of-illinois-at-chicago/student-housing/9/"
response = get(sapo, headers=headers)
print(response)

<Response [200]>


In [104]:
html_soup = BeautifulSoup(response.text, 'html.parser')

In [105]:
house_containers = html_soup.find_all('article')

In [106]:
house_containers


[<article class="premium placard" data-listingid="erd25lm" data-url="https://www.apartments.com/1345-w-fillmore-st-chicago-il-unit-3/erd25lm/">
 <section class="placardContent">
 <div class="media">
 <div class="imageContainer carousel slide" data-interval="false">
 <div class="carouselInner">
 <div class="item active" style='background-image: url("https://images1.apartments.com/i2/R7_1KaTfrvgCatFJWQ-hB5CsckooHHQjZqoBxRYBQXU/118/1345-w-fillmore-st-unit-3-chicago-il-facade.jpg");' title="facade - 1345 W Fillmore St">
 </div>
 </div>
 </div>
 <span class="listingFreshness">
 <span class="lastUpdated">
 <i class="rotateRightIcon" title="Last Updated"></i>
 <span>5 days Ago</span>
 </span>
 </span>
 </div>
 <div class="propertyInfo">
 <div class="infoPadding">
 <a class="placardTitle js-placardTitle" href="https://www.apartments.com/1345-w-fillmore-st-chicago-il-unit-3/erd25lm/" title="1345 W Fillmore St Unit 3, Chicago, IL">
 1345 W Fillmore St Unit 3</a>
 <a class="favoriteIcon neutral" 

In [260]:
titles= []
address = []
prices = []
beds=[]

In [261]:
headers = ({'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
uic_url = 'https://www.apartments.com/off-campus-housing/il/chicago/university-of-illinois-at-chicago/student-housing/28/'
r=get(uic_url, headers=headers)
page_html = BeautifulSoup(r.text, 'html.parser')
house_containers = page_html.find_all('article')
if house_containers != []:
    for container in house_containers:
        price=container.find('div', class_="apartmentRentRollupContainer")
        if(price!=None):
            price=price.span.text
            sep = ' -'
            price=price.split(sep,1)[0]
            prices.append(price)
        else:
            price=''
            prices.append(price)

            # Address
        location = container.find('a',class_="placardTitle js-placardTitle")
        if(location!=None):
            location=location.get("title")
            address.append(location)        
        else:
            location=''
            address.append(location)

            # Title 
        name=container.find('a',class_="placardTitle js-placardTitle")
        if(name!=None):
            name=name.get("title")                
            titles.append(name)
        else:
            name=''
            titles.append(name)
            
            # Beds
        bed=container.find('div', class_="apartmentRentRollupContainer")
        if(bed!=None):
            bed=bed.find('span',class_="unitLabel").text
            beds.append(bed)
        else:
            bed=''
            beds.append(bed)
        
else:
    print("nothing found")
    
    
print('You scraped {} properties.'.format(len(titles)))


You scraped 26 properties.


In [262]:
cols = ['Title', 'Price', 'Address','Beds']

properties26= pd.DataFrame({'Title': titles,
                           'Price': prices,
                           'Address':address,
                          'Beds': beds})[cols]
properties26.head()


,Title,Price,Address,Beds
0,"322 W Evergreen Ave Unit 2, Chicago, IL","$1,995","322 W Evergreen Ave Unit 2, Chicago, IL",2 Bed
1,"1513 Clybourn N Ave Unit C, Chicago, IL","$3,500","1513 Clybourn N Ave Unit C, Chicago, IL",3 Bed
2,"260 E Chestnut St Unit 1608, Chicago, IL","$1,525","260 E Chestnut St Unit 1608, Chicago, IL",Condo for Rent
3,"260 E Chestnut St Unit 44011, Chicago, IL","$2,650","260 E Chestnut St Unit 44011, Chicago, IL",Condo for Rent
4,"1630 W Le Moyne St Unit 1F, Chicago, IL","$3,995","1630 W Le Moyne St Unit 1F, Chicago, IL",Condo for Rent


In [263]:
properties26

,Title,Price,Address,Beds
0,"322 W Evergreen Ave Unit 2, Chicago, IL","$1,995","322 W Evergreen Ave Unit 2, Chicago, IL",2 Bed
1,"1513 Clybourn N Ave Unit C, Chicago, IL","$3,500","1513 Clybourn N Ave Unit C, Chicago, IL",3 Bed
2,"260 E Chestnut St Unit 1608, Chicago, IL","$1,525","260 E Chestnut St Unit 1608, Chicago, IL",Condo for Rent
3,"260 E Chestnut St Unit 44011, Chicago, IL","$2,650","260 E Chestnut St Unit 44011, Chicago, IL",Condo for Rent
4,"1630 W Le Moyne St Unit 1F, Chicago, IL","$3,995","1630 W Le Moyne St Unit 1F, Chicago, IL",Condo for Rent
5,"1442 N North Park Avenue Unit BB, Chicago, IL","$3,750","1442 N North Park Avenue Unit BB, Chicago, IL",Condo for Rent
6,"1442 N North Park Avenue Unit B, Chicago, IL","$3,750","1442 N North Park Avenue Unit B, Chicago, IL",3 Bed
7,"2231 S Sawyer Ave Unit 2, Chicago, IL","$1,325","2231 S Sawyer Ave Unit 2, Chicago, IL",Condo for Rent
8,"30 W Division St Unit 2e, Chicago, IL","$2,750","30 W Division St Unit 2e, Chicago, IL",Condo for Rent
9,"827 N Francisco Ave Unit 1, Chicago, IL","$1,900","827 N Francisco Ave Unit 1, Chicago, IL",3 Bed


In [149]:
properties.shape

(26, 4)

In [133]:
properties.drop_duplicates(inplace=True)

In [134]:
properties.reset_index(inplace=True, drop=True)

In [273]:
frames = [properties, properties1, properties2,properties3, properties4,properties5,properties6,properties7, properties8,properties9,properties10,properties11,properties12,properties13,properties14,properties15,properties16,properties17,properties18,properties19,properties20,properties21,properties22,properties23,properties24,properties25,properties26]

In [274]:
Chicago_rental= pd.concat(frames)

In [276]:
Chicago_rental.reset_index(inplace=True,drop=True)

In [277]:
Chicago_rental

,Title,Price,Address,Beds
0,Common Addams,$858,"1407 W 15th St, Chicago, IL 60608",2-4 Bed
1,"Landmark West Loop, Chicago, IL","$1,850","1035 W Van Buren St, Chicago, IL 60607",Studio - 2 Bed
2,"The Van Buren, Chicago, IL","$2,215","808 W Van Buren St, Chicago, IL 60607",Studio - 2 Bed
3,"milieu, Chicago, IL","$1,977","205 S Peoria, Chicago, IL 60607",Studio - 2 Bed
4,"the Duncan, Chicago, IL","$1,146","1515 W Monroe St, Chicago, IL 60607",Studio - 1 Bed
5,"JeffJack Apartments, Chicago, IL","$1,589","601 W Jackson Blvd, Chicago, IL 60661",Studio - 2 Bed
6,"Gateway West Loop, Chicago, IL","$1,711","11 S Green St, Chicago, IL 60607",Studio - 2 Bed
7,"Union West, Chicago, IL","$1,428","939 W Washington Blvd, Chicago, IL 60607",Studio - 3 Bed
8,"727 West Madison, Chicago, IL","$1,835","727 W Madison St, Chicago, IL 60661",Studio - 3 Bed
9,"Presidential Towers, Chicago, IL","$1,370","555 W Madison St, Chicago, IL 60661",Studio - 2 Bed


In [278]:
Chicago_rental.shape

(747, 4)

In [279]:
Chicago_rental.drop_duplicates(inplace=True)
Chicago_rental.shape

(675, 4)

In [280]:
Chicago_rental.reset_index(inplace=True,drop=True)

In [281]:
Chicago_rental

,Title,Price,Address,Beds
0,Common Addams,$858,"1407 W 15th St, Chicago, IL 60608",2-4 Bed
1,"Landmark West Loop, Chicago, IL","$1,850","1035 W Van Buren St, Chicago, IL 60607",Studio - 2 Bed
2,"The Van Buren, Chicago, IL","$2,215","808 W Van Buren St, Chicago, IL 60607",Studio - 2 Bed
3,"milieu, Chicago, IL","$1,977","205 S Peoria, Chicago, IL 60607",Studio - 2 Bed
4,"the Duncan, Chicago, IL","$1,146","1515 W Monroe St, Chicago, IL 60607",Studio - 1 Bed
5,"JeffJack Apartments, Chicago, IL","$1,589","601 W Jackson Blvd, Chicago, IL 60661",Studio - 2 Bed
6,"Gateway West Loop, Chicago, IL","$1,711","11 S Green St, Chicago, IL 60607",Studio - 2 Bed
7,"Union West, Chicago, IL","$1,428","939 W Washington Blvd, Chicago, IL 60607",Studio - 3 Bed
8,"727 West Madison, Chicago, IL","$1,835","727 W Madison St, Chicago, IL 60661",Studio - 3 Bed
9,"Presidential Towers, Chicago, IL","$1,370","555 W Madison St, Chicago, IL 60661",Studio - 2 Bed


In [288]:
nan_value = float("NaN")

Chicago_rental.replace("", nan_value, inplace=True)

Chicago_rental.dropna(axis=0,inplace=True)
Chicago_rental.shape

(668, 4)

In [292]:
Chicago_rental

,Title,Price,Address,Beds
0,Common Addams,$858,"1407 W 15th St, Chicago, IL 60608",2-4 Bed
1,"Landmark West Loop, Chicago, IL","$1,850","1035 W Van Buren St, Chicago, IL 60607",Studio - 2 Bed
2,"The Van Buren, Chicago, IL","$2,215","808 W Van Buren St, Chicago, IL 60607",Studio - 2 Bed
3,"milieu, Chicago, IL","$1,977","205 S Peoria, Chicago, IL 60607",Studio - 2 Bed
4,"the Duncan, Chicago, IL","$1,146","1515 W Monroe St, Chicago, IL 60607",Studio - 1 Bed
5,"JeffJack Apartments, Chicago, IL","$1,589","601 W Jackson Blvd, Chicago, IL 60661",Studio - 2 Bed
6,"Gateway West Loop, Chicago, IL","$1,711","11 S Green St, Chicago, IL 60607",Studio - 2 Bed
7,"Union West, Chicago, IL","$1,428","939 W Washington Blvd, Chicago, IL 60607",Studio - 3 Bed
8,"727 West Madison, Chicago, IL","$1,835","727 W Madison St, Chicago, IL 60661",Studio - 3 Bed
9,"Presidential Towers, Chicago, IL","$1,370","555 W Madison St, Chicago, IL 60661",Studio - 2 Bed


In [296]:
Chicago_rental.to_csv('Chicago_rental.csv',index=False)